# Note-taking notebook

### Image-loading with tolerance for channel number:

cv2 works for RGB and can conver to grayscale or tolerate 4 channels (Flags 1, 0 -1 respectively). This is problematic for loading images of more/less than these specified channels. 

skimage.external.tifffile functions are flexible in this extent. 



In [ ]:
import numpy as np
import cv2
from skimage import io
from skimage.external import tifffile
from resources.conv_learner import *

In [ ]:
import numpy as np
import cv2
from skimage.external import tifffile

PATH = PATH = "datasets/yeast_v1/"

testimg_cv2 = cv2.imread(PATH + 'train/WT/WT_WP_E1_S2_F1_I5_C5_A0.tif', 1)


testimg_sk1 = tifffile.imread(PATH + 'train/WT/WT_WP_E1_S2_F1_I6_C1_A0.tif')
testimg_sk2 = tifffile.imread(PATH + 'train/WT/WT_WP_E1_S2_F1_I6_C2_A0.tif')


# plt.imshow(testimg_sk)
print(testimg_cv2.shape)
print(testimg_sk1.shape)


### concatenating the two skimage-loaded images to test whether 4-channel tiff can be loaded:
chan4 = np.concatenate((testimg_sk1, testimg_sk2), axis = 0)
print(chan4.shape)

tifffile.imsave(PATH + 'train/WT/temp.tif', chan4)

testimg_fuse = tifffile.imread(PATH + 'train/WT/temp.tif')
print(testimg_fuse.shape)

In [ ]:
# testing image transformations with cv2 to find memory exhausting bug while running on yeast data with only 2 channels.

import numpy as np
import cv2
from skimage import io
from skimage.external import tifffile

PATH_yeast = "datasets/yeast_v1/"
PATH_cifar10 = "datasets/cifar10/"

test_bird_cv2 = cv2.imread(PATH_cifar10 + 'test/bird/25_bird.png').astype(np.float32)/255
test_bird_cv2_RGB = cv2.cvtColor(test_bird_cv2, cv2.COLOR_BGR2RGB)

test_yeast_io = io.imread(PATH_yeast + 'train/WT/WT_asymm.tif').astype(np.float32)/255

print(test_bird_cv2.shape)
print(test_bird_cv2_RGB.shape)
print(test_yeast_io.shape)


## io.imread loads the yeast images in order c,h,w - hence moveaxis(im, 0, -1) reorders correctly (?!)
print(np.moveaxis(test_yeast_io, 0, -1).shape)

In [ ]:
#modified open_image function tests
def imageloader(fn):
    if str(fn).lower().endswith(('.tif', '.tiff', '.tifff')):
        im = tifffile.imread(str(fn)).astype(np.float32)/255
        im = np.moveaxis(im, 0, -1)
        print('tif')
    else:
        im = cv2.imread(str(fn)).astype(np.float32)/255 
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        print('cv2')
    if im is None: raise OSError(f'File not recognized by io.imread: {fn}') 
    return im

In [ ]:
test_yeast_io = imageloader('datasets/yeast_v1/train/WT/WT_asymm.tif')
test_bird_cv2 = imageloader('datasets/cifar10/test/bird/25_bird.png')

print(test_yeast_io.shape)
print(test_bird_cv2.shape)

#### Testing image plotting with matplotlib and denorm function in tfms

In [ ]:
PATH_yeast = "datasets/yeast_v1/"
classes = ('WT', 'mfb1KO')
stats = (np.array([ 0.4914 ,  0.48216]), np.array([ 0.24703,  0.24349]))

In [ ]:
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH_yeast, val_name='test', tfms=tfms, bs=bs)

In [ ]:
data = get_data(200,4)
x,y=next(iter(data.trn_dl))

bla=data.trn_ds.denorm(x)[0,:,:,0]; print(bla.shape) #this triggers denorm() in dataset.py which includes an rollaxis()
                                                     #which returns n,sz,sz,c instead of n,c,sz,sz!

In [ ]:
input_size = [2,200,200]
x = torch.rand(3,*input_size)
print(x.shape)

### Dataset creation and normalization etc. 


In [ ]:
# Use this cell to unzip and add more data 
import zipfile

PATH = "datasets/yeast_v2/"

# with zipfile as Zip ... might help with closing issues. 
for fnames in os.listdir(PATH):
    zip_ref = zipfile.ZipFile(PATH + fnames, 'r')
    f = str(fnames).split('.')[0] #parses the zip-file name from its extension to make corresponding folder.
    os.mkdir(PATH + f)
    zip_ref.extractall(PATH + f)
    zip_ref.close()
    
print('All .zip files extracted')

In [29]:
# calculate mean and std for train, test and potentially validation dataset

import numpy as np
from skimage import io
from skimage.external import tifffile


PATH = "datasets/yeast_v2/"


for fnames in os.listdir(PATH):
    images = []
    if len(images) < 1:
        images = io.imread(os.listdir(os.path.join(PATH, fnames))[0])
    else:
        for x in os.listdir(fnames):
            images = np.stack(images, io.imread(PATH + fnames + '/' + x))

print(images.shape)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'yeast_v1_test_data'

In [34]:
fnames = os.listdir(PATH)
Check = 
print(Check)

mfb1KO_WP_E1_S2_F1_I10_C15_A0.tif


In [26]:
from resources.conv_learner import *
import numpy as np
from skimage import io
from skimage.external import tifffile

In [ ]:
from resources.conv_learner import *
PATH = "datasets/yeast_v1/"
os.makedirs(PATH,exist_ok=True)

## With pytorch ...

In [ ]:
class Yeast_Dataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:].as_matrix()
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
dataloader = torch.utils.data.DataLoader(*torch_dataset*, batch_size=140, shuffle=False, num_workers=4)

pop_mean = []
pop_std0 = []
pop_std1 = []
for i, data in enumerate(dataloader, 0):
    # shape (batch_size, 3, height, width)
    numpy_image = data['image'].numpy()
    
    # shape (3,)
    batch_mean = np.mean(numpy_image, axis=(0,2,3))
    batch_std0 = np.std(numpy_image, axis=(0,2,3))
    batch_std1 = np.std(numpy_image, axis=(0,2,3), ddof=1)
    
    pop_mean.append(batch_mean)
    pop_std0.append(batch_std0)
    pop_std1.append(batch_std1)

# shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
pop_mean = np.array(pop_mean).mean(axis=0)
pop_std0 = np.array(pop_std0).mean(axis=0)
pop_std1 = np.array(pop_std1).mean(axis=0)

In [ ]:
#testing normalize()
x = test_bird_cv2

stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

m=stats[0]
s=stats[1]

m=np.array(m, dtype=np.float32)
s=np.array(s, dtype=np.float32)

#normalize() executing code test
x = (x-m)/s

print(m)
print(s)
# print(x)

In [ ]:
import numpy as np
import cv2
from skimage import io
from skimage.external import tifffile

PATH_yeast = "datasets/yeast_v1/"
PATH_cifar10 = "datasets/cifar10/"


# testimg_sk1 = sk.io.imread(PATH_yeast + 'train/WT/WT_WP_E1_S2_F1_I6_C1_A0.tif')
# testimg_sk2 = sk.io.imread(PATH_yeast + 'train/WT/WT_WP_E1_S2_F1_I6_C2_A0.tif')

test_bird_cv2 = cv2.imread(PATH_cifar10 + 'test/bird/25_bird.png').astype(np.float32)/255

test_bird = io.imread(PATH_cifar10 + 'test/bird/25_bird.png').astype(np.float32)/255 
print(testimg_sk1.shape)
print(test_bird.shape)
print(test_bird_cv2.shape)

### concatenating the two skimage-loaded images to test whether 4-channel tiff can be loaded:

# chan4 = np.concatenate((testimg_sk1, testimg_sk2), axis = 0)
# sk.io.imsave(PATH + 'train/WT/temp.tif', chan4)
# tifffile.imsave(PATH + 'train/WT/temp.tif', chan4)

# testimg_fuse_imageio = io.imread(PATH + 'train/WT/temp.tif')
# testimg_fuse_tifffile = tifffile.imread(PATH + 'train/WT/temp.tif')

# print(testimg_fuse_imageio.shape)
# print(testimg_fuse_tifffile.shape)

### Exploring the read_dirs function in dataset.py

In [ ]:
import os

PATH = "datasets/cifar10/"
folder = 'test' #within the function network, gets passed in automatically

lbls, fnames, all_lbls = [], [], []
full_path = os.path.join(PATH, folder)
for lbl in sorted(os.listdir(full_path)):
    if lbl not in ('.ipynb_checkpoints','.DS_Store'):
        all_lbls.append(lbl)
        for fname in os.listdir(os.path.join(full_path, lbl)):
            if fname not in ('.DS_Store'):
                fnames.append(os.path.join(folder, lbl, fname))
                lbls.append(lbl)

print(lbls[0:5])
print(fnames[0:5])
print(all_lbls[0:10])
print(len(all_lbls))